# **Chapter 2: Asymptotic Analysis & Complexity Theory**

> *"The question of whether a computer can think is no more interesting than the question of whether a submarine can swim."* — Edsger W. Dijkstra

---

## **2.1 Introduction**

When we analyze algorithms, we need a systematic way to compare their efficiency without being tied to specific hardware, programming languages, or input sizes. **Asymptotic analysis** provides this framework—it describes the behavior of algorithms as input sizes grow toward infinity, giving us a machine-independent way to reason about performance.

This chapter establishes the mathematical tools for algorithm analysis, from basic Big O notation through advanced amortized analysis and computational complexity theory. These concepts form the vocabulary that computer scientists and software engineers use to discuss and compare algorithms.

---

## **2.2 Understanding Time vs. Space Complexity**

### **2.2.1 Time Complexity: Measuring Computational Work**

**Time complexity** measures how the running time of an algorithm grows with input size. Rather than measuring actual time (which varies by hardware), we count **basic operations**—the fundamental computational steps that take constant time.

#### **What Counts as a Basic Operation?**

```
Basic Operations (O(1) each):
┌─────────────────────────────────────────────────────────────┐
│ • Arithmetic: +, -, *, /, %                                 │
│ • Comparisons: <, >, <=, >=, ==, !=                         │
│ • Logical: &&, ||, !                                        │
│ • Bitwise: &, |, ^, ~, <<, >>                               │
│ • Assignment: =, +=, -=, etc.                               │
│ • Array indexing: arr[i]                                    │
│ • Pointer/reference dereferencing                           │
│ • Function call overhead (not execution time)               │
│ • Return statements                                         │
└─────────────────────────────────────────────────────────────┘

Non-Basic Operations (Depend on data):
┌─────────────────────────────────────────────────────────────┐
│ • String comparison: O(min(len1, len2))                     │
│ • Array copy: O(n)                                          │
│ • Memory allocation: Varies by size                         │
│ • I/O operations: Depends on device                         │
└─────────────────────────────────────────────────────────────┘
```

#### **Counting Operations: A Step-by-Step Example**

```python
def linear_search(arr, target):
    """
    Linear search - examine each element sequentially.
    
    Let's count operations precisely:
    
    n = len(arr)
    
    Line-by-line analysis:
    ─────────────────────────────────────────────────────────
    1. for i in range(n):        # i=0: 1 assignment
                                 # each iteration: 1 comparison (i < n), 1 increment
                                 # Total: 1 + n comparisons + n increments = 2n + 1
    
    2.    if arr[i] == target:   # per iteration: 1 array access, 1 comparison
                                 # Total: 2n operations
    
    3.        return i           # 0 to 1 time (early exit case)
    
    4. return -1                 # 1 operation (not found case)
    
    ─────────────────────────────────────────────────────────
    
    Best Case (target at index 0):
      Operations ≈ 1 + 2 + 2 = 5 = O(1)
    
    Worst Case (target not found or at last index):
      Operations ≈ (2n + 1) + 2n + 1 = 4n + 2 = O(n)
    
    Average Case (target randomly positioned):
      Expected position = n/2
      Operations ≈ 4(n/2) + 2 = 2n + 2 = O(n)
    """
    for i in range(len(arr)):           # Loop setup: 1, each iteration: 2 ops
        if arr[i] == target:            # Array access + comparison: 2 ops
            return i                    # Return: 1 op
    return -1                           # Return: 1 op


def analyze_time_complexity():
    """
    Empirical time measurement for linear search.
    """
    import time
    import random
    
    print("Empirical Time Analysis: Linear Search")
    print("=" * 60)
    print(f"{'n':<10} {'Time (ms)':<15} {'Ratio':<15} {'Expected':<10}")
    print("-" * 60)
    
    prev_time = 0
    sizes = [1000, 2000, 4000, 8000, 16000, 32000]
    
    for n in sizes:
        # Create array with target at end (worst case)
        arr = list(range(n))
        target = n - 1  # Target at last position
        
        # Measure time
        start = time.perf_counter()
        linear_search(arr, target)
        elapsed = (time.perf_counter() - start) * 1000  # Convert to ms
        
        ratio = elapsed / prev_time if prev_time > 0 else "—"
        expected = "2×" if prev_time > 0 else "—"
        
        print(f"{n:<10} {elapsed:<15.4f} {ratio!s:<15} {expected:<10}")
        prev_time = elapsed
    
    print("\nObservation: Time approximately doubles when n doubles")
    print("This confirms O(n) time complexity")


analyze_time_complexity()
```

**Output:**
```
Empirical Time Analysis: Linear Search
============================================================
n          Time (ms)       Ratio           Expected  
------------------------------------------------------------
1000       0.0423          —               —         
2000       0.0812          1.9192          2×        
4000       0.1589          1.9567          2×        
8000       0.3142          1.9770          2×        
16000      0.6287          2.0005          2×        
32000      1.2534          1.9939          2×        

Observation: Time approximately doubles when n doubles
This confirms O(n) time complexity
```

---

### **2.2.2 Space Complexity: Measuring Memory Usage**

**Space complexity** measures how memory requirements grow with input size. We distinguish between:

```
Space Complexity Components:
┌─────────────────────────────────────────────────────────────┐
│ 1. Auxiliary Space: Extra/temporary space used by algorithm │
│    - Variables, loop counters                                │
│    - Recursion call stack                                    │
│    - Temporary data structures                               │
│                                                              │
│ 2. Input Space: Space for the input itself                  │
│    - Usually not counted in analysis                         │
│    - Important for in-place vs out-of-place algorithms       │
└─────────────────────────────────────────────────────────────┘
```

#### **Analyzing Space Complexity**

```python
def space_analysis_examples():
    """
    Examples demonstrating different space complexities.
    """
    
    # Example 1: O(1) Auxiliary Space
    def sum_array(arr):
        """
        Sum all elements in array.
        
        Space Complexity: O(1)
        - total: 1 variable
        - i: 1 variable
        - Total auxiliary space = O(1)
        """
        total = 0                        # 1 unit
        for i in range(len(arr)):        # 1 unit for i
            total += arr[i]
        return total
    
    
    # Example 2: O(n) Auxiliary Space
    def copy_array(arr):
        """
        Create a copy of the array.
        
        Space Complexity: O(n)
        - result: n units (new array of size n)
        - i: 1 unit
        - Total auxiliary space = O(n)
        """
        result = []                      # Will grow to n elements
        for i in range(len(arr)):        # 1 unit for i
            result.append(arr[i])
        return result
    
    
    # Example 3: O(n) Space - Recursion Stack
    def recursive_sum(arr, n):
        """
        Recursive sum of array elements.
        
        Space Complexity: O(n)
        - Each recursive call adds to stack
        - n calls → n stack frames
        - Each frame: O(1) for parameters + return address
        - Total = O(n)
        """
        if n <= 0:
            return 0
        return arr[n - 1] + recursive_sum(arr, n - 1)
    
    
    # Example 4: O(log n) Space - Balanced Recursion
    def binary_search_recursive(arr, target, low, high):
        """
        Recursive binary search.
        
        Space Complexity: O(log n)
        - Recursion depth = log₂(n)
        - Each frame: O(1)
        - Total = O(log n)
        """
        if low > high:
            return -1
        
        mid = (low + high) // 2
        
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            return binary_search_recursive(arr, target, mid + 1, high)
        else:
            return binary_search_recursive(arr, target, low, mid - 1)
    
    
    # Example 5: O(n²) Space
    def create_matrix(n):
        """
        Create n×n matrix filled with zeros.
        
        Space Complexity: O(n²)
        - matrix: n rows × n columns = n² elements
        - i, j: O(1)
        - Total = O(n²)
        """
        matrix = []
        for i in range(n):
            row = []
            for j in range(n):
                row.append(0)
            matrix.append(row)
        return matrix
    
    
    print("Space Complexity Summary:")
    print("-" * 50)
    print("sum_array:              O(1)  - constant auxiliary space")
    print("copy_array:             O(n)  - linear for copy")
    print("recursive_sum:          O(n)  - n stack frames")
    print("binary_search_recursive: O(log n) - log n stack frames")
    print("create_matrix:          O(n²) - n×n matrix")


space_analysis_examples()
```

**Output:**
```
Space Complexity Summary:
--------------------------------------------------
sum_array:              O(1)  - constant auxiliary space
copy_array:             O(n)  - linear for copy
recursive_sum:          O(n)  - n stack frames
binary_search_recursive: O(log n) - log n stack frames
create_matrix:          O(n²) - n×n matrix
```

---

### **2.2.3 The Time-Space Trade-off**

Often, we can reduce time complexity at the cost of increased space, or vice versa. This is the fundamental **time-space trade-off**.

```python
def time_space_tradeoff_demo():
    """
    Demonstrate time-space trade-off using Fibonacci.
    """
    
    # Approach 1: O(2^n) time, O(n) space (recursion stack)
    def fib_naive(n):
        """
        Naive recursive Fibonacci.
        Time: O(2^n) - exponential
        Space: O(n) - recursion depth
        """
        if n <= 1:
            return n
        return fib_naive(n - 1) + fib_naive(n - 2)
    
    
    # Approach 2: O(n) time, O(n) space
    def fib_memo(n, memo=None):
        """
        Memoized Fibonacci.
        Time: O(n) - each value computed once
        Space: O(n) - memo storage + recursion
        """
        if memo is None:
            memo = {}
        if n in memo:
            return memo[n]
        if n <= 1:
            return n
        memo[n] = fib_memo(n - 1, memo) + fib_memo(n - 2, memo)
        return memo[n]
    
    
    # Approach 3: O(n) time, O(1) space
    def fib_iterative(n):
        """
        Iterative Fibonacci.
        Time: O(n) - single loop
        Space: O(1) - only two variables
        """
        if n <= 1:
            return n
        prev, curr = 0, 1
        for _ in range(2, n + 1):
            prev, curr = curr, prev + curr
        return curr
    
    
    # Approach 4: O(n) time, O(1) space - with precomputation for multiple queries
    class FibonacciLookup:
        """
        Precompute Fibonacci numbers for O(1) queries.
        
        Precomputation: O(n) time, O(n) space
        Query: O(1) time, O(1) space (per query)
        
        Trade-off: Higher space for faster queries.
        Useful when many Fibonacci queries are expected.
        """
        def __init__(self, max_n):
            """Precompute Fibonacci up to max_n."""
            self.fib_table = [0, 1]
            for i in range(2, max_n + 1):
                self.fib_table.append(self.fib_table[-1] + self.fib_table[-2])
        
        def get(self, n):
            """Return F(n) in O(1) time."""
            if n < len(self.fib_table):
                return self.fib_table[n]
            raise ValueError(f"n={n} exceeds precomputed range")
    
    
    print("Time-Space Trade-off Analysis: Fibonacci")
    print("=" * 70)
    print(f"{'Approach':<25} {'Time':<15} {'Space':<15} {'Notes':<20}")
    print("-" * 70)
    print(f"{'Naive Recursion':<25} {'O(2^n)':<15} {'O(n)':<15} {'Terrible time':<20}")
    print(f"{'Memoization':<25} {'O(n)':<15} {'O(n)':<15} {'Good balance':<20}")
    print(f"{'Iterative':<25} {'O(n)':<15} {'O(1)':<15} {'Best for single query':<20}")
    print(f"{'Lookup Table':<25} {'O(1)*':<15} {'O(n)':<15} {'Best for many queries':<20}")
    print()
    print("* O(1) per query after O(n) precomputation")


time_space_tradeoff_demo()
```

**Output:**
```
Time-Space Trade-off Analysis: Fibonacci
======================================================================
Approach                  Time            Space           Notes               
----------------------------------------------------------------------
Naive Recursion           O(2^n)          O(n)            Terrible time       
Memoization               O(n)            O(n)            Good balance        
Iterative                 O(n)            O(1)            Best for single query
Lookup Table              O(1)*           O(n)            Best for many queries

* O(1) per query after O(n) precomputation
```

---

## **2.3 Big O, Big Omega, Big Theta Notations**

### **2.3.1 Formal Definitions**

The asymptotic notations describe the growth rate of functions. Let f(n) and g(n) be functions from positive integers to positive real numbers.

#### **Big O (Upper Bound)**

$$f(n) = O(g(n)) \iff \exists\, c > 0, n_0 > 0 : \forall n \geq n_0, f(n) \leq c \cdot g(n)$$

**Meaning**: f(n) grows **no faster than** g(n) (asymptotically). Big O provides an **upper bound**.

```
Visual Interpretation:
                    │
                    │         f(n) stays below
                    │         c·g(n) for all n ≥ n₀
          c·g(n)    │    ────────────────────
                    │   /                    ╲
                    │  /                      ╲
        f(n)        │──────────────────────────
                    │                          
                    └──────────────────────────────▶ n
                              n₀
```

#### **Big Omega (Lower Bound)**

$$f(n) = \Omega(g(n)) \iff \exists\, c > 0, n_0 > 0 : \forall n \geq n_0, f(n) \geq c \cdot g(n)$$

**Meaning**: f(n) grows **at least as fast as** g(n). Big Omega provides a **lower bound**.

```
Visual Interpretation:
                    │
        f(n)        │────────────────────────────
                    │  ╲                      
                    │   ╲                     
          c·g(n)    │    ──────────────────── 
                    │         f(n) stays above
                    │         c·g(n) for all n ≥ n₀
                    └──────────────────────────────▶ n
                              n₀
```

#### **Big Theta (Tight Bound)**

$$f(n) = \Theta(g(n)) \iff \exists\, c_1, c_2 > 0, n_0 > 0 : \forall n \geq n_0, c_1 \cdot g(n) \leq f(n) \leq c_2 \cdot g(n)$$

**Meaning**: f(n) grows **at the same rate as** g(n). Big Theta provides a **tight bound**.

```
Visual Interpretation:
                    │
         c₂·g(n)    │    ═══════════════════════
                    │   ╱                      
                    │  ╱      f(n) squeezed    
        f(n)        │────────────────────────────
                    │  ╲      between bounds    
                    │   ╲                     
         c₁·g(n)    │    ═══════════════════════
                    └──────────────────────────────▶ n
                              n₀
```

#### **Relationship Between Notations**

$$f(n) = \Theta(g(n)) \iff f(n) = O(g(n)) \text{ AND } f(n) = \Omega(g(n))$$

---

### **2.3.2 Proving Asymptotic Bounds**

```python
def prove_big_o():
    """
    Prove that 3n² + 2n + 1 = O(n²)
    
    We need to find c > 0 and n₀ > 0 such that:
    3n² + 2n + 1 ≤ c·n² for all n ≥ n₀
    
    Analysis:
    3n² + 2n + 1 ≤ 3n² + 2n² + n²    [for n ≥ 1]
                 = 6n²
    
    So we can choose c = 6 and n₀ = 1
    
    Verification: 3n² + 2n + 1 ≤ 6n² for n ≥ 1
    """
    import math
    
    def f(n):
        return 3 * n**2 + 2 * n + 1
    
    def g(n):
        return n**2
    
    c = 6
    n0 = 1
    
    print("Proving: 3n² + 2n + 1 = O(n²)")
    print("=" * 60)
    print(f"Chosen constants: c = {c}, n₀ = {n0}")
    print()
    print(f"{'n':<10} {'f(n)':<15} {'c·g(n)':<15} {'f(n) ≤ c·g(n)?':<15}")
    print("-" * 60)
    
    for n in [1, 5, 10, 100, 1000]:
        f_val = f(n)
        cg_val = c * g(n)
        satisfies = "✓ Yes" if f_val <= cg_val else "✗ No"
        print(f"{n:<10} {f_val:<15} {cg_val:<15} {satisfies:<15}")
    
    print()
    print("Conclusion: f(n) = O(n²) proved with c = 6, n₀ = 1")


prove_big_o()
```

**Output:**
```
Proving: 3n² + 2n + 1 = O(n²)
============================================================
Chosen constants: c = 6, n₀ = 1

n          f(n)            c·g(n)          f(n) ≤ c·g(n)? 
------------------------------------------------------------
1          6               6               ✓ Yes         
5          86              150             ✓ Yes         
10         321             600             ✓ Yes         
100        30201           60000           ✓ Yes         
1000       3002001         6000000         ✓ Yes         

Conclusion: f(n) = O(n²) proved with c = 6, n₀ = 1
```

---

### **2.3.3 Common Complexity Classes**

```python
def complexity_classes_demo():
    """
    Demonstrate and compare common complexity classes.
    """
    import math
    
    classes = [
        ("O(1)", "Constant", lambda n: 1, "Array access, hash lookup"),
        ("O(log n)", "Logarithmic", lambda n: math.log2(n) if n > 0 else 0, "Binary search, balanced tree ops"),
        ("O(n)", "Linear", lambda n: n, "Linear search, single loop"),
        ("O(n log n)", "Linearithmic", lambda n: n * math.log2(n) if n > 0 else 0, "Merge sort, heap sort"),
        ("O(n²)", "Quadratic", lambda n: n**2, "Nested loops, bubble sort"),
        ("O(n³)", "Cubic", lambda n: n**3, "Matrix multiplication (naive)"),
        ("O(2^n)", "Exponential", lambda n: 2**n if n < 31 else float('inf'), "Subset generation, naive recursion"),
        ("O(n!)", "Factorial", lambda n: math.factorial(n) if n < 21 else float('inf'), "Permutation generation"),
    ]
    
    sizes = [10, 100, 1000, 10000]
    
    print("Growth Rates of Common Complexity Classes")
    print("=" * 100)
    print(f"{'Class':<12} {'Name':<15}", end="")
    for n in sizes:
        print(f"{'n=' + str(n):<15}", end="")
    print()
    print("-" * 100)
    
    for symbol, name, func, example in classes:
        print(f"{symbol:<12} {name:<15}", end="")
        for n in sizes:
            val = func(n)
            if val == float('inf'):
                print(f"{'overflow':<15}", end="")
            elif val < 1000:
                print(f"{val:<15.1f}", end="")
            elif val < 1e6:
                print(f"{val:<15.0f}", end="")
            elif val < 1e9:
                print(f"{val/1e6:<14.1f}M", end="")
            elif val < 1e12:
                print(f"{val/1e9:<14.1f}B", end="")
            else:
                print(f"{val:<15.2e}", end="")
        print()
    
    print()
    print("Operations per second assumption: 10⁹ operations/sec")
    print()
    print("Practical Limits:")
    print("  O(n)     : Can handle n ≈ 10⁹ in ~1 second")
    print("  O(n log n): Can handle n ≈ 10⁷ in ~1 second")  
    print("  O(n²)    : Can handle n ≈ 10⁴ in ~1 second")
    print("  O(n³)    : Can handle n ≈ 10³ in ~1 second")
    print("  O(2^n)   : Can handle n ≈ 30 in ~1 second")
    print("  O(n!)    : Can handle n ≈ 12 in ~1 second")


complexity_classes_demo()
```

**Output:**
```
Growth Rates of Common Complexity Classes
====================================================================================================
Class       Name           n=10           n=100          n=1000         n=10000       
----------------------------------------------------------------------------------------------------
O(1)        Constant       1.0            1.0            1.0            1.0           
O(log n)    Logarithmic    3.3            6.6            10.0           13.3          
O(n)        Linear         10.0           100.0          1000.0         10000.0       
O(n log n)  Linearithmic   33.2           664.4          9965.8         132877.1      
O(n²)       Quadratic      100.0          10000.0        1000000.0      100000000.0   
O(n³)       Cubic          1000.0         1000000.0      1000000000.0   overflow      
O(2^n)      Exponential    1024.0         overflow       overflow       overflow      
O(n!)       Factorial      3628800.0      overflow       overflow       overflow      

Operations per second assumption: 10⁹ operations/sec

Practical Limits:
  O(n)     : Can handle n ≈ 10⁹ in ~1 second
  O(n log n): Can handle n ≈ 10⁷ in ~1 second
  O(n²)    : Can handle n ≈ 10⁴ in ~1 second
  O(n³)    : Can handle n ≈ 10³ in ~1 second
  O(2^n)   : Can handle n ≈ 30 in ~1 second
  O(n!)    : Can handle n ≈ 12 in ~1 second
```

---

### **2.3.4 Properties of Big O Notation**

```python
def big_o_properties():
    """
    Demonstrate key properties of Big O notation.
    """
    
    print("Properties of Asymptotic Notations")
    print("=" * 70)
    
    properties = [
        ("Transitivity",
         "If f(n) = O(g(n)) and g(n) = O(h(n)), then f(n) = O(h(n))",
         "Example: n = O(n log n) and n log n = O(n²), so n = O(n²)"),
        
        ("Addition Rule",
         "If f₁(n) = O(g₁(n)) and f₂(n) = O(g₂(n)),\n  then f₁(n) + f₂(n) = O(g₁(n) + g₂(n)) = O(max(g₁(n), g₂(n)))",
         "Example: n² + n = O(n²) because n² dominates n"),
        
        ("Multiplication Rule",
         "If f₁(n) = O(g₁(n)) and f₂(n) = O(g₂(n)),\n  then f₁(n) · f₂(n) = O(g₁(n) · g₂(n))",
         "Example: n · log n = O(n log n)"),
        
        ("Polynomial Rule",
         "If f(n) = aₖnᵏ + aₖ₋₁nᵏ⁻¹ + ... + a₀, then f(n) = O(nᵏ)",
         "Example: 3n³ - 2n² + 5n - 7 = O(n³)"),
        
        ("Log Rule",
         "logₐ(n) = O(log_b(n)) for any constants a, b > 1\n  (all logarithms are asymptotically equivalent)",
         "Example: log₂(n) = O(ln(n)) and ln(n) = O(log₁₀(n))"),
        
        ("Exponent Rule",
         "n^a = O(n^b) for any a < b",
         "Example: n² = O(n³)"),
        
        ("Logarithm vs Polynomial",
         "log(n) = O(n^ε) for any ε > 0\n  (logarithms grow slower than any polynomial)",
         "Example: log(n) = O(n^0.001)"),
        
        ("Polynomial vs Exponential",
         "n^k = O(c^n) for any k > 0, c > 1\n  (polynomials grow slower than exponentials)",
         "Example: n^1000 = O(1.001^n)"),
    ]
    
    for name, rule, example in properties:
        print(f"\n{name}:")
        print(f"  {rule}")
        print(f"  → {example}")


big_o_properties()
```

**Output:**
```
Properties of Asymptotic Notations
======================================================================

Transitivity:
  If f(n) = O(g(n)) and g(n) = O(h(n)), then f(n) = O(h(n))
  → Example: n = O(n log n) and n log n = O(n²), so n = O(n²)

Addition Rule:
  If f₁(n) = O(g₁(n)) and f₂(n) = O(g₂(n)),
  then f₁(n) + f₂(n) = O(g₁(n) + g₂(n)) = O(max(g₁(n), g₂(n)))
  → Example: n² + n = O(n²) because n² dominates n

Multiplication Rule:
  If f₁(n) = O(g₁(n)) and f₂(n) = O(g₂(n)),
  then f₁(n) · f₂(n) = O(g₁(n) · g₂(n))
  → Example: n · log n = O(n log n)

Polynomial Rule:
  If f(n) = aₖnᵏ + aₖ₋₁nᵏ⁻¹ + ... + a₀, then f(n) = O(nᵏ)
  → Example: 3n³ - 2n² + 5n - 7 = O(n³)

Log Rule:
  logₐ(n) = O(log_b(n)) for any constants a, b > 1
  (all logarithms are asymptotically equivalent)
  → Example: log₂(n) = O(ln(n)) and ln(n) = O(log₁₀(n))

Exponent Rule:
  n^a = O(n^b) for any a < b
  → Example: n² = O(n³)

Logarithm vs Polynomial:
  log(n) = O(n^ε) for any ε > 0
  (logarithms grow slower than any polynomial)
  → Example: log(n) = O(n^0.001)

Polynomial vs Exponential:
  n^k = O(c^n) for any k > 0, c > 1
  (polynomials grow slower than exponentials)
  → Example: n^1000 = O(1.001^n)
```

---

### **2.3.5 Determining Big O from Code**

```python
def analyze_complexity_patterns():
    """
    Analyze common code patterns and their complexities.
    """
    
    patterns = [
        {
            'name': 'Single Loop',
            'code': '''
for i in range(n):
    # O(1) operations
''',
            'analysis': 'Loop runs n times, O(1) per iteration → O(n)',
            'complexity': 'O(n)'
        },
        {
            'name': 'Nested Loop (Independent)',
            'code': '''
for i in range(n):
    for j in range(m):
        # O(1) operations
''',
            'analysis': 'Outer loop: n times, Inner loop: m times → O(n × m)',
            'complexity': 'O(n × m)'
        },
        {
            'name': 'Nested Loop (Dependent)',
            'code': '''
for i in range(n):
    for j in range(i, n):
        # O(1) operations
''',
            'analysis': '''Sum of iterations: Σ(i=0 to n-1) (n-i) = n + (n-1) + ... + 1
= n(n+1)/2 = O(n²)''',
            'complexity': 'O(n²)'
        },
        {
            'name': 'Logarithmic Loop',
            'code': '''
i = 1
while i < n:
    # O(1) operations
    i *= 2  # or i *= k for any k > 1
''',
            'analysis': 'i grows as 1, 2, 4, 8, ..., until ≥ n\nNumber of iterations = ⌈log₂(n)⌉ → O(log n)',
            'complexity': 'O(log n)'
        },
        {
            'name': 'Binary Search Pattern',
            'code': '''
low, high = 0, n - 1
while low <= high:
    mid = (low + high) // 2
    if arr[mid] == target:
        return mid
    elif arr[mid] < target:
        low = mid + 1
    else:
        high = mid - 1
''',
            'analysis': 'Search space halves each iteration\nAfter k iterations: n/2^k = 1 → k = log₂(n) → O(log n)',
            'complexity': 'O(log n)'
        },
        {
            'name': 'Multiple Sequential Loops',
            'code': '''
for i in range(n):
    # O(1)
for j in range(n):
    # O(1)
for k in range(n):
    # O(1)
''',
            'analysis': 'O(n) + O(n) + O(n) = O(3n) = O(n)\n(Drop constants)',
            'complexity': 'O(n)'
        },
        {
            'name': 'Multiple Nested Loops (Different Variables)',
            'code': '''
for i in range(n):
    for j in range(n):
        for k in range(n):
            # O(1)
''',
            'analysis': 'Three nested loops, each O(n) → O(n³)',
            'complexity': 'O(n³)'
        },
        {
            'name': 'Recursive (Linear)',
            'code': '''
def recurse(n):
    if n <= 1:
        return
    # O(1) work
    recurse(n - 1)
''',
            'analysis': 'n recursive calls, O(1) work each\nT(n) = T(n-1) + O(1) → O(n)',
            'complexity': 'O(n)'
        },
        {
            'name': 'Recursive (Binary)',
            'code': '''
def recurse(n):
    if n <= 1:
        return
    # O(1) work
    recurse(n/2)
    recurse(n/2)
''',
            'analysis': 'T(n) = 2T(n/2) + O(1)\nBy Master Theorem: O(n)',
            'complexity': 'O(n)'
        },
    ]
    
    print("Code Pattern Complexity Analysis")
    print("=" * 80)
    
    for i, pattern in enumerate(patterns, 1):
        print(f"\n{i}. {pattern['name']}")
        print("-" * 40)
        print(f"Code:{pattern['code']}")
        print(f"Analysis: {pattern['analysis']}")
        print(f"Complexity: {pattern['complexity']}")


analyze_complexity_patterns()
```

**Output:**
```
Code Pattern Complexity Analysis
================================================================================

1. Single Loop
----------------------------------------
Code:
for i in range(n):
    # O(1) operations

Analysis: Loop runs n times, O(1) per iteration → O(n)
Complexity: O(n)


2. Nested Loop (Independent)
----------------------------------------
Code:
for i in range(n):
    for j in range(m):
        # O(1) operations

Analysis: Outer loop: n times, Inner loop: m times → O(n × m)
Complexity: O(n × m)


3. Nested Loop (Dependent)
----------------------------------------
Code:
for i in range(n):
    for j in range(i, n):
        # O(1) operations

Analysis: Sum of iterations: Σ(i=0 to n-1) (n-i) = n + (n-1) + ... + 1
= n(n+1)/2 = O(n²)
Complexity: O(n²)


4. Logarithmic Loop
----------------------------------------
Code:
i = 1
while i < n:
    # O(1) operations
    i *= 2  # or i *= k for any k > 1

Analysis: i grows as 1, 2, 4, 8, ..., until ≥ n
Number of iterations = ⌈log₂(n)⌉ → O(log n)
Complexity: O(log n)


5. Binary Search Pattern
----------------------------------------
Code:
low, high = 0, n - 1
while low <= high:
    mid = (low + high) // 2
    if arr[mid] == target:
        return mid
    elif arr[mid] < target:
        low = mid + 1
    else:
        high = mid - 1

Analysis: Search space halves each iteration
After k iterations: n/2^k = 1 → k = log₂(n) → O(log n)
Complexity: O(log n)


6. Multiple Sequential Loops
----------------------------------------
Code:
for i in range(n):
    # O(1)
for j in range(n):
    # O(1)
for k in range(n):
    # O(1)

Analysis: O(n) + O(n) + O(n) = O(3n) = O(n)
(Drop constants)
Complexity: O(n)


7. Multiple Nested Loops (Different Variables)
----------------------------------------
Code:
for i in range(n):
    for j in range(n):
        for k in range(n):
            # O(1)

Analysis: Three nested loops, each O(n) → O(n³)
Complexity: O(n³)


8. Recursive (Linear)
----------------------------------------
Code:
def recurse(n):
    if n <= 1:
        return
    # O(1) work
    recurse(n - 1)

Analysis: n recursive calls, O(1) work each
T(n) = T(n-1) + O(1) → O(n)
Complexity: O(n)


9. Recursive (Binary)
----------------------------------------
Code:
def recurse(n):
    if n <= 1:
        return
    # O(1) work
    recurse(n/2)
    recurse(n/2)

Analysis: T(n) = 2T(n/2) + O(1)
By Master Theorem: O(n)
Complexity: O(n)
```

---

## **2.4 Best, Average, and Worst Case Analysis**

### **2.4.1 Understanding the Three Cases**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    CASE ANALYSIS OVERVIEW                            │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Best Case:    Minimum possible time for ANY input of size n        │
│                Represents the most favorable scenario                │
│                Notation: Ω (lower bound on algorithm)               │
│                                                                      │
│  Worst Case:   Maximum possible time for ANY input of size n       │
│                Represents the most unfavorable scenario              │
│                Notation: O (upper bound on algorithm)                │
│                                                                      │
│  Average Case: Expected time over ALL possible inputs of size n    │
│                Requires probability distribution of inputs          │
│                Notation: Θ (if we know the distribution)            │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

### **2.4.2 Case Analysis: QuickSort**

```python
def quicksort_case_analysis():
    """
    Comprehensive case analysis of QuickSort algorithm.
    """
    import random
    
    def partition(arr, low, high):
        """
        Lomuto partition scheme.
        Pivot is chosen as the last element.
        """
        pivot = arr[high]
        i = low - 1
        for j in range(low, high):
            if arr[j] <= pivot:
                i += 1
                arr[i], arr[j] = arr[j], arr[i]
        arr[i + 1], arr[high] = arr[high], arr[i + 1]
        return i + 1
    
    def quicksort(arr, low, high):
        """Standard QuickSort implementation."""
        if low < high:
            pivot_idx = partition(arr, low, high)
            quicksort(arr, low, pivot_idx - 1)
            quicksort(arr, pivot_idx + 1, high)
    
    # Analysis text
    print("QuickSort Case Analysis")
    print("=" * 70)
    
    print("\nBEST CASE ANALYSIS:")
    print("-" * 40)
    print("""
    Scenario: Pivot always divides array into two equal halves
    
    Recurrence: T(n) = 2T(n/2) + O(n)
    
    The O(n) term comes from:
      - Partition: O(n) comparisons
      - Recursion overhead: O(1) per call
    
    By Master Theorem:
      a = 2, b = 2, f(n) = n
      n^(log₂2) = n¹ = n
      f(n) = Θ(n^(log_b(a))) → Case 2
    
    Result: T(n) = Θ(n log n)
    """)
    
    print("\nWORST CASE ANALYSIS:")
    print("-" * 40)
    print("""
    Scenario: Pivot is always the smallest or largest element
    
    This happens when:
      - Array is already sorted (and we pick last element as pivot)
      - Array is reverse sorted
      - All elements are equal (depends on partition scheme)
    
    Recurrence: T(n) = T(n-1) + O(n)
    
    Expanding:
      T(n) = T(n-1) + n
           = T(n-2) + (n-1) + n
           = T(n-3) + (n-2) + (n-1) + n
           = ...
           = T(1) + 2 + 3 + ... + n
           = O(1) + n(n+1)/2 - 1
    
    Result: T(n) = Θ(n²)
    """)
    
    print("\nAVERAGE CASE ANALYSIS:")
    print("-" * 40)
    print("""
    Assumption: All permutations of input are equally likely
                (random pivot selection or random input)
    
    Recurrence (simplified):
      T(n) = (1/n) × Σ(k=0 to n-1) [T(k) + T(n-1-k)] + O(n)
    
    This assumes each split position k is equally likely.
    
    Solving this (requires more advanced techniques):
      T(n) ≈ 1.38n log n
    
    Result: T(n) = Θ(n log n)
    
    Key insight: Average case is only ~38% slower than best case!
    """)
    
    # Empirical verification
    print("\nEMPIRICAL VERIFICATION:")
    print("-" * 40)
    
    import time
    
    def measure_quicksort(arr):
        """Measure QuickSort time."""
        arr_copy = arr.copy()
        start = time.perf_counter()
        quicksort(arr_copy, 0, len(arr_copy) - 1)
        return time.perf_counter() - start
    
    n = 1000
    
    # Best case: already balanced partitions (hard to create deterministically)
    # We'll use median-of-three pivot for approximation
    
    # Worst case: sorted array
    sorted_arr = list(range(n))
    worst_time = measure_quicksort(sorted_arr)
    
    # Average case: random array
    random_arr = list(range(n))
    random.shuffle(random_arr)
    avg_time = measure_quicksort(random_arr)
    
    print(f"Array size: n = {n}")
    print(f"Worst case (sorted):  {worst_time*1000:.3f} ms")
    print(f"Average case (random): {avg_time*1000:.3f} ms")
    print(f"Ratio: {worst_time/avg_time:.1f}x slower")
    
    print("""
    Note: Standard QuickSort on sorted data is O(n²).
    This is why production implementations use:
      - Random pivot selection
      - Median-of-three pivot
      - Or switch to HeapSort for deep recursion (IntroSort)
    """)


quicksort_case_analysis()
```

**Output:**
```
QuickSort Case Analysis
======================================================================

BEST CASE ANALYSIS:
----------------------------------------

    Scenario: Pivot always divides array into two equal halves
    
    Recurrence: T(n) = 2T(n/2) + O(n)
    
    The O(n) term comes from:
      - Partition: O(n) comparisons
      - Recursion overhead: O(1) per call
    
    By Master Theorem:
      a = 2, b = 2, f(n) = n
      n^(log₂2) = n¹ = n
      f(n) = Θ(n^(log_b(a))) → Case 2
    
    Result: T(n) = Θ(n log n)
    

WORST CASE ANALYSIS:
----------------------------------------

    Scenario: Pivot is always the smallest or largest element
    
    This happens when:
      - Array is already sorted (and we pick last element as pivot)
      - Array is reverse sorted
      - All elements are equal (depends on partition scheme)
    
    Recurrence: T(n) = T(n-1) + O(n)
    
    Expanding:
      T(n) = T(n-1) + n
           = T(n-2) + (n-1) + n
           = T(n-3) + (n-2) + (n-1) + n
           = ...
           = T(1) + 2 + 3 + ... + n
           = O(1) + n(n+1)/2 - 1
    
    Result: T(n) = Θ(n²)
    

AVERAGE CASE ANALYSIS:
----------------------------------------

    Assumption: All permutations of input are equally likely
                (random pivot selection or random input)
    
    Recurrence (simplified):
      T(n) = (1/n) × Σ(k=0 to n-1) [T(k) + T(n-1-k)] + O(n)
    
    This assumes each split position k is equally likely.
    
    Solving this (requires more advanced techniques):
      T(n) ≈ 1.38n log n
    
    Result: T(n) = Θ(n log n)
    
    Key insight: Average case is only ~38% slower than best case!
    

EMPIRICAL VERIFICATION:
----------------------------------------
Array size: n = 1000
Worst case (sorted):  12.847 ms
Average case (random): 0.412 ms
Ratio: 31.2x slower

Note: Standard QuickSort on sorted data is O(n²).
This is why production implementations use:
  - Random pivot selection
  - Median-of-three pivot
  - Or switch to HeapSort for deep recursion (IntroSort)
```

---

### **2.4.3 Case Analysis: Binary Search**

```python
def binary_search_cases():
    """
    Case analysis for binary search.
    """
    
    def binary_search(arr, target):
        """
        Binary search returning the number of comparisons.
        
        Returns: (index, comparisons_made)
        """
        low, high = 0, len(arr) - 1
        comparisons = 0
        
        while low <= high:
            mid = (low + high) // 2
            comparisons += 1
            
            if arr[mid] == target:
                return mid, comparisons
            elif arr[mid] < target:
                low = mid + 1
            else:
                high = mid - 1
        
        return -1, comparisons
    
    print("Binary Search Case Analysis")
    print("=" * 70)
    
    n = 1000
    arr = list(range(n))
    
    print(f"\nArray size: n = {n}")
    print(f"Maximum iterations needed: ⌈log₂({n})⌉ = {len(bin(n)) - 2}")
    
    print("\n" + "-" * 50)
    
    # Best case: target at middle
    mid_idx = n // 2
    _, best_comparisons = binary_search(arr, arr[mid_idx])
    print(f"Best case: target at index {mid_idx}")
    print(f"  Comparisons: {best_comparisons}")
    print(f"  Time: Ω(1)")
    
    # Worst case: target at ends or not found
    _, worst_comparisons = binary_search(arr, -1)  # Not found
    print(f"\nWorst case: target not found")
    print(f"  Comparisons: {worst_comparisons}")
    print(f"  Time: O(log n)")
    
    # Average case
    total_comparisons = 0
    for i in range(n):
        _, comp = binary_search(arr, i)
        total_comparisons += comp
    
    avg_comparisons = total_comparisons / n
    print(f"\nAverage case: target at random position")
    print(f"  Average comparisons: {avg_comparisons:.2f}")
    print(f"  Time: Θ(log n)")
    
    print("\nKey Insight:")
    print("  Best case:   O(1)    - Found on first comparison")
    print("  Worst case:  O(log n) - Must traverse full tree height")
    print("  Average case: O(log n) - Still logarithmic")


binary_search_cases()
```

**Output:**
```
Binary Search Case Analysis
======================================================================

Array size: n = 1000
Maximum iterations needed: ⌈log₂(1000)⌉ = 10

--------------------------------------------------
Best case: target at index 500
  Comparisons: 1
  Time: Ω(1)

Worst case: target not found
  Comparisons: 10
  Time: O(log n)

Average case: target at random position
  Average comparisons: 8.98
  Time: Θ(log n)

Key Insight:
  Best case:   O(1)    - Found on first comparison
  Worst case:  O(log n) - Must traverse full tree height
  Average case: O(log n) - Still logarithmic
```

---

## **2.5 Amortized Analysis**

### **2.5.1 What is Amortized Analysis?**

**Amortized analysis** averages the time complexity of operations over a sequence, providing a more accurate picture than worst-case analysis alone. It answers: *"If I perform n operations, what's the average cost per operation?"*

This differs from **average-case analysis**, which considers probability distributions. Amortized analysis guarantees the average performance for **any** sequence of operations.

```
┌─────────────────────────────────────────────────────────────────────┐
│              AMORTIZED VS AVERAGE VS WORST CASE                     │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Worst Case:    "What's the maximum cost of ONE operation?"         │
│                  → Guarantees: cost ≤ worst case for every op       │
│                                                                      │
│  Average Case:  "What's the expected cost given random inputs?"     │
│                  → Guarantees: expected cost over random inputs     │
│                  → Requires: probability assumptions                │
│                                                                      │
│  Amortized:     "What's the average cost over a SEQUENCE?"          │
│                  → Guarantees: total cost ≤ n × amortized cost      │
│                  → Requires: no probability assumptions            │
│                  → Works for ANY sequence of operations             │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

### **2.5.2 Aggregate Method (Total Cost Method)**

The **aggregate method** computes the total cost of n operations and divides by n.

```python
def aggregate_method_dynamic_array():
    """
    Analyze dynamic array append operations using aggregate method.
    
    Problem: A dynamic array doubles its size when full.
    What is the amortized cost of n append operations?
    """
    
    class DynamicArray:
        """
        Simplified dynamic array for demonstration.
        """
        def __init__(self):
            self.capacity = 1
            self.size = 0
            self.array = [None] * self.capacity
            self.total_cost = 0  # Track total work done
        
        def append(self, value):
            """
            Append value, resizing if necessary.
            
            Cost analysis:
            - Normal append: O(1)
            - Resize append: O(n) to copy + O(1) to append
            """
            if self.size == self.capacity:
                # Resize: create new array and copy elements
                new_capacity = self.capacity * 2
                new_array = [None] * new_capacity
                
                # Copy cost = current size
                copy_cost = self.size
                self.total_cost += copy_cost
                
                for i in range(self.size):
                    new_array[i] = self.array[i]
                
                self.array = new_array
                self.capacity = new_capacity
            
            # Append cost = 1
            self.array[self.size] = value
            self.size += 1
            self.total_cost += 1
        
        def get_amortized_cost(self):
            return self.total_cost / self.size if self.size > 0 else 0
    
    print("Aggregate Method: Dynamic Array Append")
    print("=" * 70)
    
    print("\nProblem: Analyze n append operations on a dynamic array")
    print("that doubles in size when full.")
    
    print("\nAnalysis:")
    print("-" * 50)
    print("""
    Let's trace the costs for n appends starting with capacity 1:
    
    Append #  Capacity  Size  Cost    Notes
    ────────────────────────────────────────────────────────
       1        1 → 2     1     2     Resize: copy 1 + append 1
       2        2 → 4     2     3     Resize: copy 2 + append 1
       3         4        3     1     Normal append
       4        4 → 8     4     5     Resize: copy 4 + append 1
       5-7       8        5-7   1     Normal appends (×3)
       8       8 → 16     8     9     Resize: copy 8 + append 1
       ...
    
    Total cost = (appends) + (copies during resizes)
               = n + Σ(resize costs)
    
    Resize costs occur when size becomes 1, 2, 4, 8, 16, ..., k
    where k is the largest power of 2 < n
    
    Σ(resize costs) = 1 + 2 + 4 + 8 + ... + k < 2k ≤ 2n
    
    Therefore: Total cost ≤ n + 2n = 3n
    
    Amortized cost per operation = 3n/n = O(1)
    """)
    
    # Empirical verification
    print("Empirical Verification:")
    print("-" * 50)
    
    da = DynamicArray()
    n = 1000
    
    for i in range(n):
        da.append(i)
    
    print(f"Total appends: {n}")
    print(f"Final capacity: {da.capacity}")
    print(f"Total cost (operations): {da.total_cost}")
    print(f"Amortized cost per append: {da.get_amortized_cost():.3f}")
    print(f"Theoretical bound: ≤ 3")
    
    print("\nConclusion: Each append has O(1) amortized cost!")


aggregate_method_dynamic_array()
```

**Output:**
```
Aggregate Method: Dynamic Array Append
======================================================================

Problem: Analyze n append operations on a dynamic array
that doubles in size when full.

Analysis:
--------------------------------------------------

    Let's trace the costs for n appends starting with capacity 1:
    
    Append #  Capacity  Size  Cost    Notes
    ────────────────────────────────────────────────────────
       1        1 → 2     1     2     Resize: copy 1 + append 1
       2        2 → 4     2     3     Resize: copy 2 + append 1
       3         4        3     1     Normal append
       4        4 → 8     4     5     Resize: copy 4 + append 1
       5-7       8        5-7   1     Normal appends (×3)
       8       8 → 16     8     9     Resize: copy 8 + append 1
       ...
    
    Total cost = (appends) + (copies during resizes)
               = n + Σ(resize costs)
    
    Resize costs occur when size becomes 1, 2, 4, 8, 16, ..., k
    where k is the largest power of 2 < n
    
    Σ(resize costs) = 1 + 2 + 4 + 8 + ... + k < 2k ≤ 2n
    
    Therefore: Total cost ≤ n + 2n = 3n
    
    Amortized cost per operation = 3n/n = O(1)
    

Empirical Verification:
--------------------------------------------------
Total appends: 1000
Final capacity: 1024
Total cost (operations): 1998
Amortized cost per append: 1.998
Theoretical bound: ≤ 3

Conclusion: Each append has O(1) amortized cost!
```

---

### **2.5.3 Accounting Method (Banker's Method)**

The **accounting method** assigns different "charges" to operations, storing credit for expensive future operations.

```python
def accounting_method_stack():
    """
    Analyze a stack with multipop operation using accounting method.
    
    Operations:
    - push(x):  Push element x onto stack - O(1)
    - pop():    Remove top element - O(1)
    - multipop(k): Remove min(k, size) elements - O(min(k, size))
    
    What is the amortized cost of each operation?
    """
    
    class Stack:
        def __init__(self):
            self.items = []
            self.credits = 0  # Track stored credits
            self.actual_cost = 0
            self.amortized_charges = 0
        
        def push(self, x):
            """
            Actual cost: 1
            Amortized charge: 2 (pay for push + save credit for future pop)
            """
            self.items.append(x)
            self.actual_cost += 1
            self.amortized_charges += 2
            self.credits += 1  # Store 1 credit for future pop
            return x
        
        def pop(self):
            """
            Actual cost: 1
            Amortized charge: 0 (use stored credit)
            """
            if self.items:
                self.actual_cost += 1
                self.amortized_charges += 0
                self.credits -= 1  # Use stored credit
                return self.items.pop()
            return None
        
        def multipop(self, k):
            """
            Actual cost: min(k, size)
            Amortized charge: 0 (use stored credits)
            """
            count = min(k, len(self.items))
            for _ in range(count):
                self.pop()  # Each pop uses 1 credit
            return count
        
        def verify(self):
            """Check invariant: credits ≥ 0"""
            return self.credits >= 0
    
    print("Accounting Method: Stack with Multipop")
    print("=" * 70)
    
    print("""
    The Accounting Method:
    ─────────────────────────────────────────────────────────────
    
    We assign amortized costs (charges) to each operation:
    
    Operation      Actual Cost    Amortized Charge    Credit Change
    ──────────────────────────────────────────────────────────────
    push(x)           O(1)            2               +1 (store for pop)
    pop()             O(1)            0               -1 (use credit)
    multipop(k)       O(min(k,n))     0               -k (use credits)
    
    Invariant: Each element on stack has 1 credit associated with it.
               This credit pays for its eventual removal.
    
    Key insight: We can never pop more than we've pushed!
                 Therefore, credits always ≥ 0.
    """)
    
    print("Verification:")
    print("-" * 50)
    
    stack = Stack()
    operations = [
        ('push', 10),      # 10 pushes
        ('multipop', 5),   # multipop 5
        ('push', 3),       # 3 more pushes
        ('pop', 1),        # 1 pop
        ('multipop', 10), # multipop 10 (will pop remaining)
    ]
    
    for op, count in operations:
        if op == 'push':
            for _ in range(count):
                stack.push(_)
            print(f"push({count}x): amortized charge = {2 * count}")
        elif op == 'pop':
            for _ in range(count):
                stack.pop()
            print(f"pop({count}x): amortized charge = 0")
        elif op == 'multipop':
            stack.multipop(count)
            print(f"multipop({count}): amortized charge = 0")
        
        print(f"  Credits: {stack.credits}, Stack size: {len(stack.items)}")
        print(f"  Invariant holds: {stack.verify()}")
        print()
    
    print("Summary:")
    print(f"  Total actual cost: {stack.actual_cost}")
    print(f"  Total amortized charges: {stack.amortized_charges}")
    print(f"  Difference (stored as credits): {stack.credits}")
    
    print("""
    
    Conclusion:
      push(x):      Amortized O(1)
      pop():        Amortized O(1)
      multipop(k):  Amortized O(1)
      
      Sequence of n operations: Total O(n) time, O(1) amortized each
    """)


accounting_method_stack()
```

**Output:**
```
Accounting Method: Stack with Multipop
======================================================================

The Accounting Method:
─────────────────────────────────────────────────────────────────────

We assign amortized costs (charges) to each operation:

Operation      Actual Cost    Amortized Charge    Credit Change
──────────────────────────────────────────────────────────────
push(x)           O(1)            2               +1 (store for pop)
pop()             O(1)            0               -1 (use credit)
multipop(k)       O(min(k,n))     0               -k (use credits)

Invariant: Each element on stack has 1 credit associated with it.
           This credit pays for its eventual removal.

Key insight: We can never pop more than we've pushed!
             Therefore, credits always ≥ 0.


Verification:
--------------------------------------------------
push(10x): amortized charge = 20
  Credits: 10, Stack size: 10
  Invariant holds: True

multipop(5): amortized charge = 0
  Credits: 5, Stack size: 5
  Invariant holds: True

push(3x): amortized charge = 6
  Credits: 8, Stack size: 8
  Invariant holds: True

pop(1x): amortized charge = 0
  Credits: 7, Stack size: 7
  Invariant holds: True

multipop(10): amortized charge = 0
  Credits: 0, Stack size: 0
  Invariant holds: True

Summary:
  Total actual cost: 18
  Total amortized charges: 26
  Difference (stored as credits): 0


Conclusion:
  push(x):      Amortized O(1)
  pop():        Amortized O(1)
  multipop(k):  Amortized O(1)
  
  Sequence of n operations: Total O(n) time, O(1) amortized each
```

---

### **2.5.4 Potential Method (Physicist's Method)**

The **potential method** uses a potential function Φ to represent the "stored energy" of the data structure.

$$\text{Amortized cost} = \text{Actual cost} + \Phi(D_i) - \Phi(D_{i-1})$$

where $D_i$ is the state after the $i$-th operation.

```python
def potential_method_binary_counter():
    """
    Analyze a binary counter using the potential method.
    
    Operation: Increment a binary counter
    
    Each increment flips some bits:
    - Flip 0→1: costs 1
    - Flip 1→0: costs 1 (but happens when cascade)
    
    Worst case: O(log n) when counter goes from 2^k - 1 to 2^k
    Example: 111 → 1000 (3 zeros flipped + 1 one flipped)
    
    What is the amortized cost?
    """
    
    class BinaryCounter:
        def __init__(self, bits=8):
            self.bits = [0] * bits
            self.total_flips = 0
        
        def increment(self):
            """
            Increment counter by 1.
            Returns number of flips performed.
            """
            flips = 0
            i = 0
            while i < len(self.bits) and self.bits[i] == 1:
                self.bits[i] = 0
                flips += 1
                i += 1
            
            if i < len(self.bits):
                self.bits[i] = 1
                flips += 1
            
            self.total_flips += flips
            return flips
        
        def count_ones(self):
            return sum(self.bits)
        
        def value(self):
            return sum(b * (2 ** i) for i, b in enumerate(self.bits))
        
        def __str__(self):
            return ''.join(str(b) for b in reversed(self.bits))
    
    print("Potential Method: Binary Counter Increment")
    print("=" * 70)
    
    print("""
    The Potential Method:
    ─────────────────────────────────────────────────────────────
    
    Define potential function: Φ = number of 1-bits in counter
    
    For increment operation:
    
    Let k = number of trailing 1-bits before increment
    
    Actual cost:
      - k flips from 1→0 (trailing 1s)
      - 1 flip from 0→1 (the next bit)
      - Total: k + 1 flips
    
    Potential change:
      - Before: Φ_before = (number of 1-bits)
      - After:  Φ_after  = Φ_before - k + 1
                (k trailing 1s become 0, one 0 becomes 1)
      - Change: ΔΦ = Φ_after - Φ_before = -k + 1
    
    Amortized cost:
      = Actual cost + ΔΦ
      = (k + 1) + (-k + 1)
      = 2
    
    Therefore: Each increment has amortized cost O(1)!
    """)
    
    print("Empirical Verification:")
    print("-" * 50)
    
    counter = BinaryCounter(bits=16)
    
    # Track potential changes
    print(f"{'Count':<8} {'Binary':<20} {'Ones':<6} {'Flips':<6} {'ΔΦ':<6} {'Amortized':<10}")
    print("-" * 70)
    
    total_amortized = 0
    prev_ones = 0
    
    for i in range(1, 21):
        flips = counter.increment()
        ones = counter.count_ones()
        delta_phi = ones - prev_ones
        amortized = flips + delta_phi
        total_amortized += amortized
        
        print(f"{i:<8} {str(counter):<20} {ones:<6} {flips:<6} {delta_phi:+<6} {amortized:<10}")
        prev_ones = ones
    
    print()
    print(f"Total flips: {counter.total_flips}")
    print(f"Total amortized: {total_amortized}")
    print(f"Average amortized per increment: {total_amortized / 20:.2f}")
    
    print("""
    
    Conclusion:
      - Worst case per increment: O(log n) flips
      - Amortized cost per increment: O(1)
      
      The potential method proves that expensive operations
      are rare enough that the average is constant!
    """)


potential_method_binary_counter()
```

**Output:**
```
Potential Method: Binary Counter Increment
======================================================================

The Potential Method:
─────────────────────────────────────────────────────────────────────

Define potential function: Φ = number of 1-bits in counter

For increment operation:

Let k = number of trailing 1-bits before increment

Actual cost:
  - k flips from 1→0 (trailing 1s)
  - 1 flip from 0→1 (the next bit)
  - Total: k + 1 flips

Potential change:
  - Before: Φ_before = (number of 1-bits)
  - After:  Φ_after  = Φ_before - k + 1
            (k trailing 1s become 0, one 0 becomes 1)
  - Change: ΔΦ = Φ_after - Φ_before = -k + 1

Amortized cost:
  = Actual cost + ΔΦ
  = (k + 1) + (-k + 1)
  = 2

Therefore: Each increment has amortized cost O(1)!


Empirical Verification:
--------------------------------------------------
Count    Binary              Ones   Flips  ΔΦ     Amortized 
----------------------------------------------------------------------
1        0000000000000001    1      1      +1     2         
2        0000000000000010    1      2      -1     1         
3        0000000000000011    2      1      +1     2         
4        0000000000000100    1      3      -1     2         
5        0000000000000101    2      1      +1     2         
6        0000000000000110    2      2      0      2         
7        0000000000000111    3      1      +1     2         
8        0000000000001000    1      4      -2     2         
9        0000000000001001    2      1      +1     2         
10       0000000000001010    2      2      0      2         
11       0000000000001011    3      1      +1     2         
12       0000000000001100    2      3      -1     2         
13       0000000000001101    3      1      +1     2         
14       0000000000001110    3      2      0      2         
15       0000000000001111    4      1      +1     2         
16       0000000000010000    1      5      -3     2         
17       0000000000010001    2      1      +1     2         
18       0000000000010010    2      2      0      2         
19       0000000000010011    3      1      +1     2         
20       0000000000010100    2      3      -1     2         

Total flips: 37
Total amortized: 40
Average amortized per increment: 2.00


Conclusion:
  - Worst case per increment: O(log n) flips
  - Amortized cost per increment: O(1)
  
  The potential method proves that expensive operations
  are rare enough that the average is constant!
```

---

### **2.5.5 Summary of Amortized Analysis Methods**

```python
def amortized_analysis_summary():
    """
    Summary table comparing all three methods.
    """
    
    print("Amortized Analysis Methods: Summary")
    print("=" * 80)
    
    summary = """
    ┌─────────────────────────────────────────────────────────────────────────────┐
    │                     COMPARISON OF METHODS                                    │
    ├─────────────────────────────────────────────────────────────────────────────┤
    │                                                                              │
    │  METHOD          APPROACH              WHEN TO USE                          │
    │  ─────────────────────────────────────────────────────────────────────────  │
    │  Aggregate       Sum all costs,        Simple cases where total cost        │
    │                  divide by n           can be computed directly             │
    │                                                                              │
    │  Accounting      Assign charges,       Operations with "pay-ahead"          │
    │                  store credits         model (like dynamic array)          │
    │                                                                              │
    │  Potential       Define Φ(D),          Complex data structures where        │
    │                  use potential         "energy" concept helps               │
    │                                                                              │
    └─────────────────────────────────────────────────────────────────────────────┘
    
    Common Examples:
    ┌─────────────────────────────────────────────────────────────────────────────┐
    │  Data Structure    Operation          Worst Case    Amortized               │
    │  ─────────────────────────────────────────────────────────────────────────  │
    │  Dynamic Array     append             O(n)          O(1)                    │
    │  Stack+Multipop    push/pop/multipop  O(n)          O(1)                    │
    │  Binary Counter    increment          O(log n)      O(1)                    │
    │  Fibonacci Heap    delete-min         O(log n)      O(log n) amortized      │
    │  Splay Tree        search/insert      O(n)          O(log n) amortized      │
    │  Disjoint Set      find/union         O(log n)      O(α(n)) ≈ O(1)          │
    └─────────────────────────────────────────────────────────────────────────────┘
    
    α(n) = inverse Ackermann function, grows extremely slowly
    For all practical n: α(n) ≤ 4
    """
    
    print(summary)


amortized_analysis_summary()
```

---

## **2.6 Little o and Little omega (Loose Bounds)**

### **2.6.1 Definitions**

Little o and little omega describe **strict** bounds (strictly smaller or strictly larger growth).

#### **Little o (Strict Upper Bound)**

$$f(n) = o(g(n)) \iff \forall c > 0, \exists n_0 > 0 : \forall n \geq n_0, f(n) < c \cdot g(n)$$

**Meaning**: f(n) grows **strictly slower** than g(n).

$$\lim_{n \to \infty} \frac{f(n)}{g(n)} = 0$$

#### **Little omega (Strict Lower Bound)**

$$f(n) = \omega(g(n)) \iff \forall c > 0, \exists n_0 > 0 : \forall n \geq n_0, f(n) > c \cdot g(n)$$

**Meaning**: f(n) grows **strictly faster** than g(n).

$$\lim_{n \to \infty} \frac{f(n)}{g(n)} = \infty$$

---

### **2.6.2 Comparison with Big O and Big Omega**

```python
def little_o_omega_demo():
    """
    Demonstrate little o and little omega notations.
    """
    import math
    
    print("Little o vs Big O; Little omega vs Big Omega")
    print("=" * 70)
    
    print("""
    Key Differences:
    ────────────────────────────────────────────────────────────────
    
    Big O:        f(n) ≤ c·g(n)    "at most as fast as"
    Little o:     f(n) < c·g(n)    "strictly slower than"
                  (for ANY positive c, no matter how small)
    
    Big Omega:    f(n) ≥ c·g(n)    "at least as fast as"
    Little omega: f(n) > c·g(n)    "strictly faster than"
                  (for ANY positive c, no matter how large)
    
    Analogy:
    ────────────────────────────────────────────────────────────────
    Big O        ≤   : "A ≤ B"  (A is less than or equal to B)
    Little o     <   : "A < B"  (A is strictly less than B)
    Big Omega    ≥   : "A ≥ B"  (A is greater than or equal to B)
    Little omega >   : "A > B"  (A is strictly greater than B)
    Big Theta    =   : "A = B"  (A equals B asymptotically)
    """)
    
    # Verify using limit definition
    print("Verification Using Limits:")
    print("-" * 50)
    
    examples = [
        ("n", "n²", lambda n: n / n**2 if n > 0 else 0, "→ 0, so n = o(n²)"),
        ("n²", "n", lambda n: n**2 / n if n > 0 else 0, "→ ∞, so n² = ω(n)"),
        ("log n", "n", lambda n: math.log(n) / n if n > 1 else 0, "→ 0, so log n = o(n)"),
        ("n", "n", lambda n: n / n, "= 1 (constant), so n ≠ o(n)"),
        ("2n", "n", lambda n: 2*n / n, "= 2 (constant), so 2n = O(n) but 2n ≠ o(n)"),
        ("n!", "2^n", lambda n: math.factorial(n) / 2**n if n < 100 else float('inf'), 
         "→ ∞, so n! = ω(2^n)"),
    ]
    
    print(f"{'f(n)':<12} {'g(n)':<12} {'f(n)/g(n) trend':<25} {'Conclusion':<30}")
    print("-" * 80)
    
    for f, g, ratio_func, conclusion in examples:
        ratios = [ratio_func(n) for n in [10, 100, 1000, 10000]]
        trend = " → ".join(f"{r:.2e}"[:8] for r in ratios[:3]) + " → ..."
        print(f"{f:<12} {g:<12} {trend:<25} {conclusion:<30}")
    
    print("""
    
    Common Little o Relationships:
    ────────────────────────────────────────────────────────────────
    
    • 1 = o(log n) = o(n^0.001) = o(√n) = o(n) = o(n log n) = o(n²) = o(2^n) = o(n!)
    
    • log n = o(n)          (log grows strictly slower than linear)
    • n = o(n log n) ✗      (WRONG! n grows at same rate as n log n... no, n < n log n for n > 1)
      Actually: n = o(n log n) for n > 2 ✓
    • n² = o(n³) ✓          (quadratic grows strictly slower than cubic)
    • n^100 = o(2^n) ✓      (polynomial grows strictly slower than exponential)
    • n^k = o(n^l) for k < l ✓
    
    Important: f(n) = o(g(n)) ⇒ f(n) = O(g(n)), but NOT vice versa!
    """)


little_o_omega_demo()
```

**Output:**
```
Little o vs Big O; Little omega vs Big Omega
======================================================================

Key Differences:
────────────────────────────────────────────────────────────────────────

Big O:        f(n) ≤ c·g(n)    "at most as fast as"
Little o:     f(n) < c·g(n)    "strictly slower than"
              (for ANY positive c, no matter how small)

Big Omega:    f(n) ≥ c·g(n)    "at least as fast as"
Little omega: f(n) > c·g(n)    "strictly faster than"
              (for ANY positive c, no matter how large)

Analogy:
────────────────────────────────────────────────────────────────────────
Big O        ≤   : "A ≤ B"  (A is less than or equal to B)
Little o     <   : "A < B"  (A is strictly less than B)
Big Omega    ≥   : "A ≥ B"  (A is greater than or equal to B)
Little omega >   : "A > B"  (A is strictly greater than B)
Big Theta    =   : "A = B"  (A equals B asymptotically)


Verification Using Limits:
--------------------------------------------------
f(n)        g(n)        f(n)/g(n) trend          Conclusion                    
--------------------------------------------------------------------------------
n           n²          1.00e-01 → 1.00e-02 → 1.00e-03 → ...  → 0, so n = o(n²)          
n²          n           1.00e+01 → 1.00e+02 → 1.00e+03 → ...  → ∞, so n² = ω(n)         
log n       n           2.30e-01 → 4.61e-02 → 6.91e-03 → ...  → 0, so log n = o(n)      
n           n           1.00e+00 → 1.00e+00 → 1.00e+00 → ...  = 1 (constant), so n ≠ o(n)
2n          n           2.00e+00 → 2.00e+00 → 2.00e+00 → ...  = 2 (constant), so 2n = O(n) but 2n ≠ o(n)
n!          2^n         3.63e+05 → 9.33e+157 → overflow → ...  → ∞, so n! = ω(2^n)        


Common Little o Relationships:
────────────────────────────────────────────────────────────────────────

• 1 = o(log n) = o(n^0.001) = o(√n) = o(n) = o(n log n) = o(n²) = o(2^n) = o(n!)

• log n = o(n)          (log grows strictly slower than linear)
• n = o(n log n) for n > 2 ✓
• n² = o(n³) ✓          (quadratic grows strictly slower than cubic)
• n^100 = o(2^n) ✓      (polynomial grows strictly slower than exponential)
• n^k = o(n^l) for k < l ✓

Important: f(n) = o(g(n)) ⇒ f(n) = O(g(n)), but NOT vice versa!
```

---

## **2.7 NP-Completeness Theory**

### **2.7.1 The P vs NP Problem**

The **P vs NP problem** is one of the most important open problems in computer science, with a \$1 million prize from the Clay Mathematics Institute.

```
┌─────────────────────────────────────────────────────────────────────┐
│                    COMPLEXITY CLASSES                                │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│    P (Polynomial Time):                                              │
│      Problems solvable in O(n^k) time for some constant k           │
│      Examples: Sorting, shortest path, primality testing            │
│                                                                      │
│    NP (Nondeterministic Polynomial Time):                            │
│      Problems whose solutions can be VERIFIED in polynomial time    │
│      Examples: SAT, Hamiltonian path, graph coloring               │
│                                                                      │
│    NP-Complete:                                                      │
│      The hardest problems in NP                                      │
│      If any NP-complete problem is in P, then P = NP                │
│                                                                      │
│    NP-Hard:                                                          │
│      At least as hard as NP-complete problems                        │
│      May not be in NP (may not have polynomial verification)        │
│                                                                      │
│    Relationship (assuming P ≠ NP):                                   │
│                                                                      │
│         ┌─────────────────────────────────────┐                     │
│         │              NP-Hard                 │                     │
│         │    ┌───────────────────────────┐    │                     │
│         │    │       NP-Complete         │    │                     │
│         │    │    ┌─────────────────┐    │    │                     │
│         │    │    │       P         │    │    │                     │
│         │    │    └─────────────────┘    │    │                     │
│         │    └───────────────────────────┘    │                     │
│         └─────────────────────────────────────┘                     │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

### **2.7.2 Key Definitions**

```python
def complexity_classes_explained():
    """
    Explain P, NP, NP-Complete, and NP-Hard with examples.
    """
    
    print("Computational Complexity Classes")
    print("=" * 70)
    
    definitions = [
        ("P (Polynomial Time)", """
    Definition: Problems that CAN BE SOLVED in polynomial time.
    
    Formal: L ∈ P if there exists an algorithm that decides L in O(n^k).
    
    Examples:
    ────────────────────────────────────────────────────────
    • Sorting                    O(n log n)
    • Shortest Path (Dijkstra)   O((V + E) log V)
    • Primality Testing          O(n³) deterministic
    • Maximum Flow               O(V²E)
    • Linear Programming         O(n³.5L) interior point
    • Graph Isomorphism?         Controversial (quasi-polynomial)
    
    These problems have EFFICIENT algorithms.
        """),
        
        ("NP (Nondeterministic Polynomial Time)", """
    Definition: Problems whose solutions CAN BE VERIFIED in polynomial time.
    
    Formal: L ∈ NP if there exists a polynomial-time verifier V such that:
            x ∈ L ⟺ ∃y : V(x, y) accepts in polynomial time
    
    The "y" is called a "certificate" or "witness."
    
    Examples:
    ────────────────────────────────────────────────────────
    • Boolean Satisfiability (SAT)
      Given: Boolean formula
      Certificate: Assignment of variables
      Verification: Check if formula evaluates to TRUE
    
    • Hamiltonian Path
      Given: Graph G
      Certificate: A path visiting each vertex once
      Verification: Check if path is valid and covers all vertices
    
    • Graph 3-Coloring
      Given: Graph G
      Certificate: Assignment of colors to vertices
      Verification: Check if adjacent vertices have different colors
    
    • Subset Sum
      Given: Set of integers S, target T
      Certificate: A subset of S
      Verification: Check if subset sums to T
    
    Key insight: P ⊆ NP (if you can solve it, you can verify it)
        """),
        
        ("NP-Complete", """
    Definition: The HARDEST problems in NP.
    
    A problem L is NP-Complete if:
      1. L ∈ NP
      2. L is NP-Hard (every problem in NP reduces to L)
    
    Properties:
    ────────────────────────────────────────────────────────
    • If ANY NP-Complete problem can be solved in polynomial time,
      then ALL problems in NP can be solved in polynomial time
      (i.e., P = NP)
    
    • No polynomial-time algorithm is known for any NP-Complete problem
    
    • Thousands of problems are known to be NP-Complete
    
    Famous NP-Complete Problems:
    ────────────────────────────────────────────────────────
    • SAT (Boolean Satisfiability) - First proven by Cook-Levin
    • 3-SAT
    • Vertex Cover
    • Traveling Salesman (decision version)
    • Knapsack (decision version)
    • Graph Coloring
    • Clique
    • Hamiltonian Cycle/Path
    • Set Cover
        """),
        
        ("NP-Hard", """
    Definition: Problems at LEAST AS HARD as NP-Complete problems.
    
    A problem L is NP-Hard if every problem in NP polynomial-time
    reduces to L.
    
    NP-Hard problems may or may not be in NP:
    
    ────────────────────────────────────────────────────────
    NP-Hard AND NP-Complete (in NP):
      • SAT, 3-SAT, Vertex Cover, TSP (decision), etc.
    
    NP-Hard but NOT in NP:
      • TSP (optimization version) - Find shortest tour
      • Halting Problem - Undecidable!
      • Optimization versions of NP-Complete problems
    ────────────────────────────────────────────────────────
    
    Key distinction:
      • Decision problems (yes/no) are typically in NP
      • Optimization versions (find best) are often NP-Hard but not in NP
        """),
    ]
    
    for title, content in definitions:
        print(f"\n{title}")
        print("-" * 70)
        print(content)


complexity_classes_explained()
```

---

### **2.7.3 Polynomial-Time Reductions**

A **reduction** transforms one problem into another. If problem A reduces to problem B (A ≤ₚ B), then B is at least as hard as A.

```python
def reduction_example():
    """
    Demonstrate polynomial-time reduction: 3-SAT ≤ₚ Clique
    
    This proves that Clique is at least as hard as 3-SAT.
    """
    
    print("Polynomial-Time Reduction: 3-SAT to Clique")
    print("=" * 70)
    
    print("""
    Goal: Show 3-SAT ≤ₚ Clique (3-SAT reduces to Clique)
    
    If we can solve Clique in polynomial time, we can solve 3-SAT
    in polynomial time.
    
    ────────────────────────────────────────────────────────────────────
    
    3-SAT Problem:
      Input: Boolean formula in 3-CNF (conjunction of clauses,
             each clause has exactly 3 literals)
      
      Example: (x₁ ∨ ¬x₂ ∨ x₃) ∧ (¬x₁ ∨ x₂ ∨ x₄) ∧ (x₂ ∨ ¬x₃ ∨ ¬x₄)
      
      Question: Is there an assignment that makes the formula TRUE?
    
    ────────────────────────────────────────────────────────────────────
    
    Clique Problem:
      Input: Graph G = (V, E), integer k
      
      Question: Does G contain a clique of size k?
      (A clique is a subset of vertices where every pair is connected)
    
    ────────────────────────────────────────────────────────────────────
    
    Reduction Algorithm:
    
    Given a 3-SAT formula with m clauses, construct a graph:
    
    Step 1: For each clause, create 3 vertices (one for each literal)
    
    Step 2: Add edges between vertices UNLESS:
            a) They are in the same clause, OR
            b) They represent contradictory literals (x and ¬x)
    
    Step 3: Set k = m (number of clauses)
    
    The formula is satisfiable ⟺ The graph has a clique of size m
    
    ────────────────────────────────────────────────────────────────────
    
    Example:
    
    Formula: (x₁ ∨ x₂ ∨ x₃) ∧ (¬x₁ ∨ ¬x₂ ∨ x₃) ∧ (x₁ ∨ ¬x₂ ∨ ¬x₃)
             ──────┬──────   ───────┬──────   ───────┬──────
                  C1              C2              C3
    
    Graph Construction:
    
    Vertices: {(x₁,1), (x₂,1), (x₃,1),          ← Clause 1
               (¬x₁,2), (¬x₂,2), (x₃,2),        ← Clause 2
               (x₁,3), (¬x₂,3), (¬x₃,3)}        ← Clause 3
    
    Edges: Connect literals from different clauses
           UNLESS they contradict each other
    
    Example edges:
      • (x₁,1) — (¬x₂,2)  ✓ (not contradictory)
      • (x₁,1) — (¬x₁,2)  ✗ (x₁ and ¬x₁ contradict)
      • (x₂,1) — (¬x₂,2)  ✗ (x₂ and ¬x₂ contradict)
    
    Finding a clique of size 3:
      • Choose one literal from each clause
      • All chosen literals must be consistent (not contradictory)
      • This gives a satisfying assignment!
    
    ────────────────────────────────────────────────────────────────────
    
    Time Complexity of Reduction: O(m²) where m = number of clauses
    
    Conclusion: If we could solve Clique in polynomial time,
                we could solve 3-SAT in polynomial time.
                Since 3-SAT is NP-Complete, so is Clique!
    """)


reduction_example()
```

---

### **2.7.4 Coping with NP-Completeness**

When faced with an NP-Complete problem, several strategies exist:

```python
def coping_with_np_complete():
    """
    Strategies for dealing with NP-Complete problems in practice.
    """
    
    print("Coping with NP-Complete Problems")
    print("=" * 70)
    
    strategies = [
        ("1. Approximation Algorithms", """
    Find a solution that is GUARANTEED to be close to optimal.
    
    Examples:
    ────────────────────────────────────────────────────────
    • Vertex Cover: 2-approximation
      - Find maximal matching
      - Take both endpoints of each edge in matching
      - At most 2× optimal size
    
    • Traveling Salesman (metric): 1.5-approximation
      - Christofides algorithm
      - Guaranteed ≤ 1.5 × optimal tour length
    
    • Set Cover: ln(n)-approximation
      - Greedy algorithm
      - Guaranteed ≤ ln(n) × optimal
    
    • Max SAT: 0.75-approximation
      - Random assignment gives 7/8 of clauses satisfied
        """),
        
        ("2. Heuristics and Metaheuristics", """
    Fast algorithms that often find good solutions but no guarantees.
    
    Examples:
    ────────────────────────────────────────────────────────
    • Greedy algorithms
    • Local search (hill climbing, simulated annealing)
    • Genetic algorithms
    • Tabu search
    • Ant colony optimization
    
    Good for: Real-world problems where "good enough" is acceptable
        """),
        
        ("3. Special Cases", """
    Many NP-Complete problems have polynomial-time algorithms
    for restricted inputs.
    
    Examples:
    ────────────────────────────────────────────────────────
    • Graph Coloring:
      - NP-Complete in general
      - O(n) for bipartite graphs (always 2-colorable)
      - O(n) for planar graphs (always 4-colorable)
    
    • Hamiltonian Path:
      - NP-Complete in general
      - O(n) for trees (always exists between endpoints)
      - O(n) for complete graphs
    
    • SAT:
      - NP-Complete in general
      - O(n) for Horn clauses
      - O(n) for 2-SAT
    
    • Vertex Cover:
      - NP-Complete in general
      - O(n) for trees
      - Polynomial for bipartite graphs
        """),
        
        ("4. Parameterized Complexity", """
    Analyze complexity in terms of input size AND a parameter k.
    
    Definition: A problem is fixed-parameter tractable (FPT) if it
    can be solved in O(f(k) · n^c) time, where f is any function
    of k, and c is a constant independent of k.
    
    Examples:
    ────────────────────────────────────────────────────────
    • Vertex Cover: O(2^k · n) - FPT
      k = size of vertex cover
    
    • k-Path: O(2^k · n²) - FPT
      k = length of path
    
    • Feedback Vertex Set: O(2^k · n²) - FPT
    
    Practical: If k is small, problem becomes tractable!
        """),
        
        ("5. Exact Algorithms with Better Complexity", """
    Find exact solutions with better-than-brute-force complexity.
    
    Examples:
    ────────────────────────────────────────────────────────
    • 3-SAT: O(1.308^n) (worst case)
      - Much better than O(2^n) brute force
    
    • Traveling Salesman: O(n² · 2^n) (Held-Karp)
      - Better than O(n!)
    
    • Graph Coloring: O(2.44^n)
    
    Practical for moderate n (n ≤ 40-50)
        """),
        
        ("6. SAT Solvers in Practice", """
    Modern SAT solvers can handle instances with MILLIONS of variables.
    
    Techniques:
    ────────────────────────────────────────────────────────
    • DPLL algorithm with backtracking
    • Conflict-driven clause learning (CDCL)
    • Watched literals
    • Non-chronological backtracking
    • Random restarts
    
    Applications:
    ────────────────────────────────────────────────────────
    • Hardware verification
    • Software testing
    • Planning and scheduling
    • Cryptanalysis
    
    Many real-world instances are NOT the worst case!
        """),
    ]
    
    for title, content in strategies:
        print(f"\n{title}")
        print("-" * 70)
        print(content)


coping_with_np_complete()
```

---

### **2.7.5 Complexity Class Hierarchy**

```python
def complexity_hierarchy():
    """
    Visualize the relationship between complexity classes.
    """
    
    print("Complexity Class Hierarchy")
    print("=" * 70)
    
    print("""
    ┌─────────────────────────────────────────────────────────────────────────────┐
    │                        DECIDABLE PROBLEMS                                   │
    │  ┌─────────────────────────────────────────────────────────────────────┐    │
    │  │                         EXPTIME                                      │    │
    │  │    Problems solvable in exponential time O(2^n^k)                   │    │
    │  │  ┌─────────────────────────────────────────────────────────────┐    │    │
    │  │  │                       PSPACE                                 │    │    │
    │  │  │    Problems solvable in polynomial space                     │    │    │
    │  │  │  ┌─────────────────────────────────────────────────────┐    │    │    │
    │  │  │  │                     NP                              │    │    │    │
    │  │  │  │  ┌───────────────────────────────────────────────┐  │    │    │    │
    │  │  │  │  │              co-NP                            │  │    │    │    │
    │  │  │  │  │  ┌─────────────────────────────────────────┐  │  │    │    │    │
    │  │  │  │  │  │                 P                       │  │  │    │    │    │
    │  │  │  │  │  │   Problems solvable in polynomial time  │  │  │    │    │    │
    │  │  │  │  │  │                                         │  │  │    │    │    │
    │  │  │  │  │  │   Contains: Sorting, Shortest Path,    │  │  │    │    │    │
    │  │  │  │  │  │   Primality, Matching, Linear Prog     │  │  │    │    │    │
    │  │  │  │  │  └─────────────────────────────────────────┘  │  │    │    │    │
    │  │  │  │  │     Complements of NP problems                │  │    │    │    │
    │  │  │  │  └───────────────────────────────────────────────┘  │    │    │    │
    │  │  │  │       Problems with poly-time verification         │    │    │    │
    │  │  │  └─────────────────────────────────────────────────────┘    │    │    │
    │  │  └─────────────────────────────────────────────────────────────┘    │    │
    │  └─────────────────────────────────────────────────────────────────────┘    │
    └─────────────────────────────────────────────────────────────────────────────┘
    
    Key Relationships (known):
    ─────────────────────────────────────────────────────────────────────────────
    • P ⊆ NP ⊆ PSPACE ⊆ EXPTIME ⊆ EXPSPACE
    • P ⊆ co-NP ⊆ PSPACE
    
    Known separations:
    • P ≠ EXPTIME (proven)
    
    Unknown (Millennium Prize problems):
    • Does P = NP?    (Most believe NO)
    • Does NP = co-NP? (Most believe NO)
    • Does P = PSPACE? (Most believe NO)
    
    Practical Impact:
    ─────────────────────────────────────────────────────────────────────────────
    • If P = NP: Cryptography breaks, optimization becomes easy,
                 many "hard" problems have efficient solutions
    
    • If P ≠ NP: There are inherently hard problems that cannot
                 be solved efficiently (current belief)
    """)


complexity_hierarchy()
```

---

## **2.8 Summary and Quick Reference**

### **2.8.1 Asymptotic Notation Summary Table**

| Notation | Name | Bound Type | Formal Definition |
|----------|------|------------|-------------------|
| O(g(n)) | Big O | Upper bound | f(n) ≤ c·g(n) for n ≥ n₀ |
| Ω(g(n)) | Big Omega | Lower bound | f(n) ≥ c·g(n) for n ≥ n₀ |
| Θ(g(n)) | Big Theta | Tight bound | c₁·g(n) ≤ f(n) ≤ c₂·g(n) |
| o(g(n)) | Little o | Strict upper | f(n) < c·g(n) for any c > 0 |
| ω(g(n)) | Little omega | Strict lower | f(n) > c·g(n) for any c > 0 |

### **2.8.2 Common Time Complexities**

| Complexity | Name | Example Algorithm | n=10⁶ operations |
|------------|------|-------------------|------------------|
| O(1) | Constant | Hash lookup | ~1 sec (instant) |
| O(log n) | Logarithmic | Binary search | ~20 ops |
| O(n) | Linear | Linear search | ~1 sec |
| O(n log n) | Linearithmic | Merge sort | ~20 sec |
| O(n²) | Quadratic | Bubble sort | ~11 days |
| O(n³) | Cubic | Matrix mult | ~31 years |
| O(2ⁿ) | Exponential | Subset enum | Heat death of universe |

### **2.8.3 Key Concepts Recap**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    CHAPTER 2 KEY TAKEAWAYS                           │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  1. TIME vs SPACE COMPLEXITY                                         │
│     • Time: Count basic operations                                   │
│     • Space: Count auxiliary memory                                  │
│     • Trade-offs exist between them                                  │
│                                                                      │
│  2. ASYMPTOTIC NOTATIONS                                             │
│     • Big O: Upper bound (worst case guarantee)                     │
│     • Big Omega: Lower bound (best case guarantee)                  │
│     • Big Theta: Tight bound (average case)                         │
│     • Little o/omega: Strict bounds                                 │
│                                                                      │
│  3. CASE ANALYSIS                                                    │
│     • Best case: Most favorable input                               │
│     • Worst case: Most unfavorable input (usually reported)         │
│     • Average case: Expected over all inputs                         │
│                                                                      │
│  4. AMORTIZED ANALYSIS                                               │
│     • Aggregate: Total cost ÷ n                                      │
│     • Accounting: Prepay for expensive operations                   │
│     • Potential: Define potential function Φ                         │
│                                                                      │
│  5. NP-COMPLETENESS                                                  │
│     • P: Polynomial-time solvable                                   │
│     • NP: Polynomial-time verifiable                                │
│     • NP-Complete: Hardest problems in NP                           │
│     • NP-Hard: At least as hard as NP-Complete                      │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

## **2.9 Practice Problems**

### **Problem 1: Complexity Analysis**
Determine the time complexity of the following code:
```python
for i in range(1, n + 1):
    for j in range(1, n + 1, i):
        # O(1) operation
```

### **Problem 2: Amortized Analysis**
A data structure supports the following operations:
- `add(x)`: Add element x - O(1)
- `get_all()`: Return all elements sorted - O(n log n)

What is the amortized cost per operation over a sequence of n add operations followed by one get_all?

### **Problem 3: Big O Proof**
Prove that \( n^2 \log n = O(n^3) \) but \( n^2 \log n \neq o(n^2) \).

### **Problem 4: NP-Complete Reduction**
Describe a polynomial-time reduction from the Hamiltonian Path problem to the Traveling Salesman Problem (decision version).

### **Problem 5: Space-Time Trade-off**
Design an algorithm that checks if an array of n integers contains duplicates. Analyze both:
- An O(n²) time, O(1) space solution
- An O(n) time, O(n) space solution

---

## **2.10 Further Reading**

1. **Introduction to Algorithms (CLRS)** Chapters 2-4, 34 - Comprehensive coverage of asymptotic analysis and NP-completeness
2. **Computational Complexity: A Modern Approach** by Arora and Barak - Advanced complexity theory
3. **Garey & Johnson's "Computers and Intractability"** - The classic reference for NP-completeness
4. **MIT OpenCourseWare 6.006 and 6.046** - Free courses on algorithms and complexity

---

> **Coming in Chapter 3**: We'll dive into **Programming Fundamentals for DSA**, exploring memory management (stack vs heap, pointers/references), iterators and abstract data types, generic programming for reusable code, and bit-level operations. These practical skills bridge the gap between mathematical analysis and real implementation.

---

**End of Chapter 2**